In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Implementation of NeuralNetwork

    -Hidden layer:2 and input and output layer

In [10]:
class NeuralNetwork:
    
    def __init__(self,input_size,layers_dim,output_size):
        np.random.seed(0)
        
        models = {}  #it store the weight and bias of every layer(hiddden,input and output layer)
        
        #Input layer and bias--1st hidden layer weights
        models["W1"]=np.random.randn(input_size,layers_dim[0]) #(n X l0)
        models["b1"]=np.zeros((1,layers_dim[0]))#(1 X l0)
        
        #1stHL-2ndHL wts and bias
        models["W2"] = np.random.randn(layers_dim[0],layers_dim[1])#(l0 X l1)
        models["b2"]=np.zeros((1,layers_dim[2]))#(1 X l1)
        
        #2ndHL-output layer wts
        models["W3"] = np.random.randn(layers_dim[1],output_size)#(l1 X k)
        models["b3"]=np.zeros((1,output_size))#(1 X k)
        
        self.models=models
        self.activations_=None
        
    @staticmethod    
    def softmax(z):
        exp_z=np.exp(z)  #elementwise operation #vector
        
        return exp_z/np.sum(exp_z,axis=1,keepdims=True)
    
    def forward(self,x):
        W1 , W2 , W3 = self.models["W1"] , self.models["W2"] , self.models["W3"]
        b1 , b2 , b3 = self.models["b1"] , self.models["b2"] , self.models["b3"]
        
        Z1 = np.dot(x,W1) + b1    #(m x n) X (n X l0) -- (m X l0)
        A1 = np.tanh(Z1)
        
        Z2 = np.dot(A1,W2) + b2   #(m x l0) X (l0 X l1) -- (m X l1)
        A2 = np.tanh(Z2)
        
        Z3 = np.dot(A2,W3) + b3    #(m x l1) X (l1 X l2) -- (m X l2)
        Y_ = NeuralNetwork.softmax(Z3)
        
        self.activations_ = A1 , A2 , Y_
        
        return Y_
    
    def backward(self,x,y,alpha=0.001): #Here y is one hot vector of (m X k)
        
        W1 , W2 , W3 = self.models["W1"] , self.models["W2"] , self.models["W3"]
        b1 , b2 , b3 = self.models["b1"] , self.models["b2"] , self.models["b3"]
        
        A1 , A2 , Y_ = self.activations_
        
        
        # 1)_Gradient Calculation 
        
        #loss propagate from output layer to 2nd HL
        delta_3 = Y_ - y  #(m X k)
        dW3 = np.dot(A2.T,delta_3) #A2.T--(l1 X m) #delta_3--(m X k) #(A2.T X delta_3)--(dW3)--(l1 X k)
        db3 = np.sum(delta_3,axis=0,keepdims=True)/x.shape[0] #delta_3--(m X k) db3--(1Xk)
        
        #(1-A2^2) is derivative of tanh function 
        #loss propagate from 2nd HL to 1st HL
        delta_2 = delta_3.dot(W3.T) * (1-np.square(A2)) #((m X k) X (k X l1)) * (m Xl1))---(m X l1)
        dw2 = np.dot(A1.T,delta_2)# ((l0 X m) X (m X l1)) --- (l0 X l1)
        db2 = np.sum(delta_2,axis=0,keepdims=True)/x.shape[0] #(m X l1)---(1 X l1)
        
        #loss propagate from 1st HL to input layer 
        delta_1 = delta_2.dot(W2.T) * (1-np.square(A1)) #( ((m X l1) X (l1 X l0)) * (m X l0) ) --- (m X l0)
        dw1 = np.dot(x.T,delta_1) #( (n X m) X (m X l0) ) --- (n X l0)
        db1 = np.sum(delta_1,axis=0,keepdims=True)/x.shape[0] #(m X l0) --- (1 X l0)
        
        # 2)_Update Weights
        
        self.models["w1"] -= alpha * dw1
        self.models["w2"] -= alpha * dw2
        self.models["w3"] -= alpha * dw3
        
        self.models["b1"] -= alpha * db1
        self.models["b2"] -= alpha * db2
        self.models["b3"] -= alpha * db3
        
    def predict(self,x):
        
        return np.argmax(self.forward(x),axis=1)
    
    @staticmethod
    def loss(y_label,y_predict):#y_label is one hot vector means only one idx is 1(repr. class)
                                 # and other is 0 -----Categorical Cross Entropy E(m example)
        return -np.mean(y_label*np.log(y_predict)) #np.sum and np.mean both are right
    
    @staticmethod
    def one_hot_vector(y,k):#k=len(Unique class)
        
        y_ohv=np.zeros((y.shape[0],k))
        y_ohv[np.arange(y.shape[0]),y]=1
        
        return y_ohv
    
    @staticmethod
    def fit(X,Y,alpha=0.001,epoch=50):
        
        loss=[]
        
        # 1)-Create the one hot vector
        
        k=len(np.unique(Y))
        y_ohv=NeuralNetwork.one_hot_vector(Y,k)
        
        #2)-Calculate loss of every forward propagate and backpropagate the loss 
        
        for i in range(epoch):
            y_=self.forward(X)
            l=self.loss(Y,y_)
            loss.append(l)
            self.backward(X,Y,alpha)
            
            #3)print the loss of every 5 epoch
            
            if i%5==0:
                print("{} epoch --> Loss is {} ".format(i,l))
    
        
        return loss

    def accuracy(self,X,Y):
        pred=self.predict(X)
        return np.sum(Y==pred)/X.shape[0]
        

In [11]:
NeuralNetwork(4,np.array([4,3,4]),3).softmax

<function __main__.NeuralNetwork.softmax(z)>

In [ ]:
!git add Implementation_NNScratch.ipynb

In [ ]:
!git commit -m"NN 2 hidden layer"